0. 추천시스템의 개념과 목적을 이해한다. \
1. Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다. \
2. 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다. \
3. 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다 \
4. 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.\
5. 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

In [1]:
import pandas as pd
import os

fname = 'C:/Users/minho/aiffel/datasets/lastfm-dataset-360K/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [2]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [3]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [5]:
# 유저 수
data['user_id'].nunique()

358868

In [6]:
# 아티스트 수
data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [8]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [9]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

In [ ]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Greenday' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

In [15]:
data[data.isin({'artist':['charlie']})['artist']]

,user_id,artist,play
703761,0a45d397a995abdac1c7a0a607348cd4dc72292b,charlie,5
1105860,1012f7870db6c6f7b304211533c13833680bf46d,charlie,62
1178157,112d3b62956be8d9ea554ca2c5271a617d1e74e3,charlie,531
1205756,11981a3e9d260ea0f6176936cdb32ab8a15c7f26,charlie,493
1264347,12721aa36c929e84b0d7584cde4782a75d69ea83,charlie,46
1589356,173166072c9783b949501f33e049931556e0a7ea,charlie,139
1736220,195aa9030703b88cea0f53499b253c25c40d0b7c,charlie,99
1771402,19da102f108cf3eb703b7060208d05339bdd7be3,charlie,255
1941757,1c5e9b1ee4a89b713a597138d2983d8d4dff3314,charlie,40
2010943,1d5563345c0c077c3d6dd249f5805acbaac12b1b,charlie,22


In [19]:
data[data['artist'].str.contains('puth')]
# ValueError: Cannot mask with non-boolean array containing NA / NaN values
# (참고) pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [21]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['greenday' , 'maroon5' ,'jason mraz' ,'coldplay', 'boyslikegirls']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,greenday,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,boyslikegirls,30


In [22]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [23]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [24]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 



temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,85635,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


In [25]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


In [29]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

In [26]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [27]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [30]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [31]:
# 모델 훈련
als_model.fit(csr_data_transpose)

In [32]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [33]:
zimin_vector

array([-0.15325463,  0.63052976, -0.05186073,  0.36638847, -0.3742061 ,
        0.19379625,  0.67352515,  0.84561104,  0.11733968, -0.01774785,
        0.05802066,  0.01467218, -0.7247721 , -0.0787935 , -0.16713056,
       -0.44024718,  0.0053319 , -0.06851988,  0.2924234 ,  0.5071505 ,
        0.769692  , -0.2051107 ,  0.9179265 ,  0.6627185 , -0.40806282,
        0.30852354, -0.36251196, -0.20454597,  0.3630716 , -0.52936864,
       -0.29960272, -0.7906194 ,  0.18669379, -0.4813955 , -0.5270405 ,
       -0.8000974 ,  0.1724058 ,  0.8480523 , -0.615115  , -0.528438  ,
       -1.2117372 ,  0.3181025 ,  0.309586  ,  0.39729413, -0.05671247,
        0.04680888,  0.420035  ,  0.1212293 , -0.1844194 , -0.07369551,
        0.22328258,  0.65351856, -0.03634469, -0.4541512 ,  0.202658  ,
       -0.30020308,  0.04186001, -0.6977524 , -0.60891575,  0.06354914,
        0.5436641 ,  0.02730005,  0.06612121,  0.34488586, -0.79495585,
        0.27406228,  0.15820089, -0.06018568,  0.30648026, -0.41

In [34]:
black_eyed_peas_vector

array([-2.96662445e-03,  1.76313519e-02,  1.01933377e-02, -1.16595542e-02,
        1.39893973e-02,  1.78413428e-02,  3.05646798e-04, -3.79210309e-04,
       -5.37058944e-03,  1.26894144e-02,  2.04824917e-02,  6.19243644e-03,
        1.10070892e-02,  8.20336305e-03, -1.15225064e-02, -1.34168081e-02,
       -5.61278872e-03,  4.29212349e-03,  1.12731280e-02,  2.63644243e-03,
        3.36299697e-03, -3.63847893e-03, -8.07083398e-03,  3.63988080e-03,
       -4.20407066e-03,  1.66851003e-02,  1.63372606e-02,  1.57552815e-04,
       -1.39455376e-02,  1.83025487e-02,  5.02677262e-03, -9.83589888e-03,
        1.48273502e-02, -1.62230972e-02,  1.33898994e-02, -2.08926690e-03,
        1.28039047e-02,  1.41449925e-02,  2.41107978e-02,  2.77880859e-02,
       -5.64519502e-03,  2.09931973e-02, -8.01637187e-04,  2.59771254e-02,
        1.40115013e-02,  1.99166611e-02,  4.44567204e-03,  3.58138961e-04,
       -3.65323795e-04,  1.16933873e-02, -1.16558652e-02,  2.62378575e-03,
        8.13757349e-03,  

In [35]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.12307818

In [36]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.2626857

In [37]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.99999994),
 (277, 0.98744076),
 (28, 0.98124206),
 (5, 0.97737694),
 (217, 0.9765435),
 (473, 0.9747312),
 (247, 0.967975),
 (418, 0.9668855),
 (490, 0.9622961),
 (694, 0.9548868),
 (910, 0.9537902),
 (268, 0.9512093),
 (1018, 0.9492811),
 (55, 0.9479786),
 (782, 0.9428739)]

In [38]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'placebo',
 'the beatles',
 'u2',
 'oasis',
 'foo fighters',
 'nirvana',
 'pink floyd',
 'the smashing pumpkins',
 'arctic monkeys',
 'the white stripes']

In [39]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [40]:
get_similar_artist('2pac')

['2pac',
 'jay-z',
 'snoop dogg',
 'nas',
 'the game',
 'notorious b.i.g.',
 'dr. dre',
 '50 cent',
 'ludacris',
 'ice cube']

In [41]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(28, 0.30165467),
 (24, 0.30052286),
 (247, 0.29814965),
 (277, 0.29075122),
 (5, 0.28882527),
 (217, 0.28624684),
 (418, 0.2823009),
 (490, 0.28192002),
 (55, 0.2714829),
 (262, 0.26888338),
 (75, 0.26268572),
 (473, 0.26255387),
 (268, 0.2613919),
 (694, 0.25899512),
 (969, 0.25548512),
 (910, 0.2545708),
 (35, 0.25420088),
 (509, 0.25338167),
 (627, 0.2508684),
 (457, 0.2489356)]

In [42]:
[idx_to_artist[i[0]] for i in artist_recommended]

['the killers',
 'jack johnson',
 'the beatles',
 'muse',
 'red hot chili peppers',
 'radiohead',
 'u2',
 'oasis',
 'arctic monkeys',
 'snow patrol',
 'queen',
 'placebo',
 'pink floyd',
 'foo fighters',
 'kings of leon',
 'nirvana',
 'john mayer',
 'keane',
 'maroon 5',
 'franz ferdinand']

In [45]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)
explain

(0.1270340671473212,
 [(3746, 0.06859126426439437),
  (62, 0.05931269310153843),
  (259000, -0.00021391896823976599),
  (85635, -0.00024355524359744447),
  (270115, -0.00041241600677435167)],
 (array([[ 2.87003432,  2.25423617,  2.14755472, ...,  2.38895991,
           2.36626742,  2.2262256 ],
         [ 6.46973517,  1.1687335 ,  0.51629721, ...,  0.5441401 ,
           0.51682117,  0.47224715],
         [ 6.16355574,  5.44450937,  1.02216434, ...,  0.31590629,
           0.31646887,  0.25827243],
         ...,
         [ 6.85639693,  6.02123462,  5.73426829, ...,  0.83906493,
           0.03610724, -0.01038616],
         [ 6.7912687 ,  5.93815182,  5.67200528, ...,  6.35798684,
           0.85316754,  0.04315412],
         [ 6.38934388,  5.57036935,  5.28875806, ...,  5.88115411,
           5.88285186,  0.81534205]]),
  False))

In [44]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('jason mraz', 0.06859126426439437),
 ('coldplay', 0.05931269310153843),
 ('boyslikegirls', -0.00021391896823976599),
 ('greenday', -0.00024355524359744447),
 ('maroon5', -0.00041241600677435167)]